In [ ]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt

import pickle

In [ ]:
case_ind=[0,3,4,5,6,7,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30]+[i for i in range(31,44)]+[i for i in range(47,60)]+[100,101,102,103,104,105,106]
# 4,5,6,17,19,26,46 -- 10,41
# 35,11 -- 0,31

# 48, rename gemm as case 100, [30,110,3]
# 49
# 50
# 51, rename kernal_2mm as case 103, [15,88,3]
# 52, rename kernel_durbin_smaller as case 104, [8,109,4]
# 53
fp=open('multiplication_index','rb')
multiplication_index=pickle.load(fp)

### Convert solutions generated by RL to json files

In [ ]:
case_ind=[0,3,4,5,6,7,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30]+[i for i in range(31,44)]+[i for i in range(47,60)]+[100,101,102,103,104,105,106]
# 4,5,6,17,19,26,46 -- 10,41
# 35,11 -- 0,31

# 48, rename gemm as case 100, [30,110,3]
# 49
# 50
# 51, rename kernal_2mm as case 103, [15,88,3]
# 52, rename kernel_durbin_smaller as case 104, [8,109,4]
# 53
fp=open('multiplication_index','rb')

multiplication_index=pickle.load(fp)

cases=[ 5, 11, 17, 26, 35, 48, 51, 53, 54]
# 7, 18, 24, 35, 47,100,103,105,106
# 5, 11, 17, 26, 35, 48, 51, 53, 54
# 0,  1,  2,  3,  4,  5,  6,  7,  8

fp=open('RL_solution/solution_96_gamma_95','rb')
rl=pickle.load(fp)
c_dict_solution=rl[1]

In [ ]:
# synthetic DFGs
cases_index=0
graph_index=cases[cases_index]
current_mul=multiplication_index[graph_index]
solution_len=len(current_mul)

solution=[]
target=[]

dsp_low=0
dsp_up=200

for dsp_target in range(dsp_low,dsp_up):
    if (graph_index,dsp_target) in c_dict_solution:
        current_solution_set=c_dict_solution[graph_index,dsp_target]
        for j in range(len(current_solution_set)): # j th solution in solution set
            current_solution=[]
            if len(current_solution_set[j])==solution_len:
                for k in range(solution_len):
                    if current_solution_set[j][k]==1:
                        current_solution=current_solution+[current_mul[k]+1]
                solution=solution+[current_solution]
            else:
                modified_solution=current_solution_set[j]+[1 for i in range(int(solution_len-len(current_solution_set[j])))]
                for k in range(solution_len):
                    if modified_solution[k]==1:
                        current_solution=current_solution+[current_mul[k]+1]
                solution=solution+[current_solution]
            target=target+[dsp_target]
            
# save to json files
import pandas as pd
df=pd.DataFrame([target,solution],index=["target_DSP", "solution"])
df.to_json(r'RL_solution/mu_1/case_'+str(case_ind[graph_index])+'_RL_solution_mu_1.json')

In [ ]:
# real DFGs
cases_index=5
graph_index=cases[cases_index]
current_mul=multiplication_index[graph_index]
solution_len=len(current_mul)

fp=open('info_case_'+str(case_ind[graph_index])+'_node_number_mapping','rb')
mapping=pickle.load(fp)
inv_map = {v: k for k, v in mapping.items()}


solution=[]
target=[]

dsp_low=0
dsp_up=200

for dsp_target in range(dsp_low,dsp_up):
    if (graph_index,dsp_target) in c_dict_solution:
        current_solution_set=c_dict_solution[graph_index,dsp_target]
        for j in range(len(current_solution_set)): # j th solution in solution set
            current_solution=[]
            if len(current_solution_set[j])==solution_len:
                for k in range(solution_len):
                    if current_solution_set[j][k]==1:
                        current_solution=current_solution+[inv_map[current_mul[k]]+1]
                solution=solution+[current_solution]
            else:
                modified_solution=current_solution_set[j]+[1 for i in range(int(solution_len-len(current_solution_set[j])))]
                for k in range(solution_len):
                    if modified_solution[k]==1:
                        current_solution=current_solution+[inv_map[current_mul[k]]+1]
                solution=solution+[current_solution]
            target=target+[dsp_target]
            
# save to json files
import pandas as pd
df=pd.DataFrame([target,solution],index=["target_DSP", "solution"])
df.to_json(r'RL_solution/mu_1/case_'+str(case_ind[graph_index])+'_RL_solution_mu_1.json')
df.to_json(r'RL_solution/mu_1/case_'+str(case_ind[graph_index])+'_RL_solution_mu_1.json')

### Convert solutions generated by RL w FT to json files

In [ ]:
# data load for mu_1 (with FT)
fp=open('RL_solution/mu_1/mu_1_solution_case_7_graph_index_5','rb')
case_7=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_18_graph_index_11','rb')
case_18=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_24_graph_index_17','rb')
case_24=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_35_graph_index_26','rb')
case_35=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_47_graph_index_35','rb')
case_47=pickle.load(fp)


fp=open('RL_solution/mu_1/mu_1_solution_case_100_graph_index_48','rb')
case_100=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_103_graph_index_51','rb')
case_103=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_105_graph_index_53','rb')
case_105=pickle.load(fp)

fp=open('RL_solution/mu_1/mu_1_solution_case_106_graph_index_54','rb')
case_106=pickle.load(fp)



cases_mark=[case_7[0],case_18[0],case_24[0],case_35[0],case_47[0],\
    case_100[0],case_103[0],case_105[0],case_106[0]]
cases_set=[case_7[1],case_18[1],case_24[1],case_35[1],case_47[1],\
    case_100[1],case_103[1],case_105[1],case_106[1]]
cases=[ 5, 11, 17, 26, 35, 48, 51, 53, 54]
# 7, 18, 24, 35, 47,100,103,105,106
# 5, 11, 17, 26, 35, 48, 51, 53, 54
# 0,  1,  2,  3,  4,  5,  6,  7,  8

In [ ]:
# synthetic DFGs
cases_index=0
graph_index=cases[cases_index]
current_mul=multiplication_index[graph_index]
solution_len=len(current_mul)


c_dict_mark=cases_mark[cases_index]
c_dict_solution=cases_set[cases_index]


solution=[]
target=[]

dsp_low=0
dsp_up=200

for dsp_target in range(dsp_low,dsp_up):
    if (graph_index,dsp_target) in c_dict_solution:
        current_solution_set=c_dict_solution[graph_index,dsp_target]
        for j in range(len(current_solution_set)): # j th solution in solution set
            current_solution=[]
            if len(current_solution_set[j])==solution_len:
                for k in range(solution_len):
                    if current_solution_set[j][k]==1:
                        current_solution=current_solution+[current_mul[k]+1]
                solution=solution+[current_solution]
            else:
                modified_solution=current_solution_set[j]+[1 for i in range(int(solution_len-len(current_solution_set[j])))]
                for k in range(solution_len):
                    if modified_solution[k]==1:
                        current_solution=current_solution+[current_mul[k]+1]
                solution=solution+[current_solution]
            target=target+[dsp_target]

# save to json files
import pandas as pd
df=pd.DataFrame([target,solution],index=["target_DSP", "solution"])
df.to_json(r'RL_solution/mu_1/case_'+str(case_ind[graph_index])+'_RL_solution_mu_1.json')

In [ ]:
# real DFGs
cases_index=5
graph_index=cases[cases_index]
current_mul=multiplication_index[graph_index]
solution_len=len(current_mul)


c_dict_mark=cases_mark[cases_index]
c_dict_solution=cases_set[cases_index]


fp=open('info_case_'+str(case_ind[graph_index])+'_node_number_mapping','rb')
mapping=pickle.load(fp)
inv_map = {v: k for k, v in mapping.items()}


solution=[]
target=[]

dsp_low=0
dsp_up=200

for dsp_target in range(dsp_low,dsp_up):
    if (graph_index,dsp_target) in c_dict_solution:
        current_solution_set=c_dict_solution[graph_index,dsp_target]
        for j in range(len(current_solution_set)): # j th solution in solution set
            current_solution=[]
            if len(current_solution_set[j])==solution_len:
                for k in range(solution_len):
                    if current_solution_set[j][k]==1:
                        current_solution=current_solution+[inv_map[current_mul[k]]+1]
                solution=solution+[current_solution]
            else:
                modified_solution=current_solution_set[j]+[1 for i in range(int(solution_len-len(current_solution_set[j])))]
                for k in range(solution_len):
                    if modified_solution[k]==1:
                        current_solution=current_solution+[inv_map[current_mul[k]]+1]
                solution=solution+[current_solution]
            target=target+[dsp_target]

# save to json files
import pandas as pd
df=pd.DataFrame([target,solution],index=["target_DSP", "solution"])
df.to_json(r'RL_solution/mu_1/case_'+str(case_ind[graph_index])+'_RL_solution_mu_1.json')

### Convert solutions generated by SA/GA/PSA/ACO to json files

In [ ]:
case_ind=[0,3,4,5,6,7,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30]+[i for i in range(31,44)]+[i for i in range(47,60)]+[100,101,102,103,104,105,106]
# 5,17,26-- 10,41
# 35,11 -- 0,31

# 48, rename gemm as case 100, [30,110,3]
# 49
# 50
# 51, rename kernal_2mm as case 103, [15,88,3]
# 52, rename kernel_durbin_smaller as case 104, [8,109,4]
# 53 [8,87,3]
# 54 [18,165,5]
fp=open('multiplication_index','rb')
multiplication_index=pickle.load(fp)

In [ ]:
# synthetic cases
name='mu_1'
dsp_low=10
dsp_up=41

inter=1
solution=[]

graph_index=5
current_mul=multiplication_index[graph_index]

target=[int(np.floor(i/3)) for i in range(3*dsp_low,3*dsp_up)]

for dsp_target in range(dsp_low,dsp_up,inter):
    fp=open('GA/GA_'+name+'_graph_'+str(graph_index)+'_dsp_'+str(dsp_target),'rb')
    current_ga=pickle.load(fp)
    for j in range(3):
        current_solution=[]
        for k in range(len(current_ga[0])):
            if current_ga[j][k]==1:
                current_solution=current_solution+[current_mul[k]+1]
        solution=solution+[current_solution]
        
# for mu
import pandas as pd
df=pd.DataFrame([target,solution],index=["target_DSP", "solution"])

df.to_json(r'GA/'+name+'/case_'+str(case_ind[graph_index])+'_GA_solution_'+name+'.json')

In [ ]:
# for real cases
dsp_low=30
dsp_up=110

inter=3
solution=[]

graph_index=48
current_mul=multiplication_index[graph_index]

fp=open('info_case_'+str(case_ind[graph_index])+'_node_number_mapping','rb')
mapping=pickle.load(fp)
inv_map = {v: k for k, v in mapping.items()}

target=[]
for i in range(dsp_low,dsp_up,inter):
    target=target+[i,i,i]

for dsp_target in range(dsp_low,dsp_up,inter):
    fp=open('GA/GA_graph_'+str(graph_index)+'_dsp_'+str(dsp_target),'rb')
    current_ga=pickle.load(fp)
    for j in range(len(current_ga)):
        current_solution=[]
        for k in range(len(current_ga[0])):
            if current_ga[j][k]==1:
                current_solution=current_solution+[inv_map[current_mul[k]]+1]
        solution=solution+[current_solution]
        
# for mu
import pandas as pd
df=pd.DataFrame([target,solution],index=["target_DSP", "solution"])

df.to_json(r'GA/'+name+'/case_'+str(case_ind[graph_index])+'_GA_solution_'+name+'.json')